### Métricas de clasificación

Reutilizamos el cuaderno anterior.

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('Data.csv')

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

df.columns = df.columns.str.lower().str.replace(' ', '_')

cadena_columnas = list(df.dtypes[df.dtypes == 'object'].index)

for col in cadena_columnas:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.churn = (df.churn == 'yes').astype(int)

Modelo completo

In [ ]:
df_train_completo, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_completo, test_size=0.33, random_state=11)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [ ]:
categoricas = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numericas = ['tenure', 'monthlycharges', 'totalcharges']


In [ ]:
train_dict = df_train[categoricas + numericas].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

modeloRL = LogisticRegression(solver='liblinear', random_state=1)
modeloRL.fit(X_train, y_train)

In [ ]:
val_dict = df_val[categoricas + numericas].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred= modeloRL.predict_proba(X_val)[:, 1]

Un modelo más pequeño para desarrollar.

In [ ]:
subconjunto = ['contract', 'tenure', 'totalcharges']
train_dict_sub = df_train[subconjunto].to_dict(orient='records')
dv_sub = DictVectorizer(sparse=False)
dv_sub.fit(train_dict_sub)

X_sub_train = dv_sub.transform(train_dict_sub)

submodeloRL = LogisticRegression(solver='liblinear', random_state=1)
submodeloRL.fit(X_sub_train, y_train)

In [ ]:
sub_val_dict = df_val[subconjunto].to_dict(orient='records')
X_sub_val = dv_sub.transform(sub_val_dict)

sub_y_pred = submodeloRL.predict_proba(X_sub_val)[:, 1]

#### Exactitud

Exactitud para el modelo completo.

In [ ]:
y_pred = modeloRL.predict_proba(X_val)[:, 1]
churn = y_pred >= 0.5
(churn == y_val).mean()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_val, y_pred >= 0.5)

Exactitud para el modelo más pequeño

In [ ]:
#val_dict_sub = df_val[subconjunto].to_dict(orient='records')
#X_sub_val = dv_sub.transform(val_dict_sub)
#sub_y_pred= submodeloRL.predict_proba(X_sub_val)[:, 1]
#sub_churn = sub_y_pred >= 0.5
#accuracy_score(y_val, sub_churn)

In [ ]:
umbrales = np.linspace(0, 1, 11)
umbrales

In [ ]:
umbrales = np.linspace(0, 1, 21)

exactitudes = []

for t in umbrales:
    acc = accuracy_score(y_val, y_pred >= t)
    exactitudes.append(acc)
    print('%0.2f %0.3f' % (t, acc))

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(umbrales,exactitudes, color='black')

plt.title('Umbral vs Exactitud')
plt.xlabel('Umbral')
plt.ylabel('Exactitud')

plt.xticks(np.linspace(0, 1, 11))

plt.show()

In [ ]:
sub_churn = sub_y_pred >= 0.5
(sub_churn == y_val).mean()

In [ ]:
accuracy_score(y_val, sub_churn)

Exactitud para un baseline inicial

In [ ]:
size_val = len(y_val)
baseline = np.repeat(False, size_val)
baseline

Ahora podemos verificar la exactitud de esta predicción del baseline usando el mismo código que usamos anteriormente: 


In [ ]:
accuracy_score(baseline, y_val)

#### Tabla de confusión

In [ ]:
verdadero_positivo = ((y_pred >= 0.5) & (y_val == 1)).sum()
falso_positivo = ((y_pred >= 0.5) & (y_val == 0)).sum()
falso_negativo = ((y_pred < 0.5) & (y_val == 1)).sum()
verdadero_negativo = ((y_pred < 0.5) & (y_val == 0)).sum()

In [ ]:
tabla_confusion = np.array(
     # predict neg    pos
    [[verdadero_negativo, falso_positivo], # actual neg
     [falso_negativo, verdadero_positivo]]) # actual pos

tabla_confusion

In [ ]:
tabla_confusion / tabla_confusion.sum()

Tabla de confusión para el submodelo propuesto.

In [ ]:
verdadero_positivo = ((sub_y_pred >= 0.5) & (y_val == 1)).sum()
falso_positivo = ((sub_y_pred >= 0.5) & (y_val == 0)).sum()
falso_negativo = ((sub_y_pred < 0.5) & (y_val == 1)).sum()
verdadero_negativo = ((sub_y_pred < 0.5) & (y_val == 0)).sum()

In [ ]:
tabla_confusion = np.array(
     # predict neg    pos
    [[verdadero_negativo, falso_positivo], # actual neg
     [falso_negativo, verdadero_positivo]]) # actual pos

tabla_confusion

In [ ]:
tabla_confusion / tabla_confusion.sum()

#### Precisión y exhaustividad

In [ ]:
precision = verdadero_positivo / (verdadero_positivo + falso_positivo)
exhaustividad =  verdadero_positivo / (verdadero_positivo + falso_negativo)
precision, exhaustividad

In [ ]:
tabla_confusion / tabla_confusion.sum()

In [ ]:
precision = verdadero_positivo / (verdadero_positivo + falso_positivo)
exhaustividad = verdadero_positivo / (verdadero_positivo + falso_negativo)
precision, exhaustividad

#### ROC y AUC

TPR y FPR

In [ ]:
puntuaciones = []

umbrales = np.linspace(0, 1, 101)

for t in umbrales: #B
    tp = ((y_pred >= t) & (y_val == 1)).sum()
    fp = ((y_pred >= t) & (y_val == 0)).sum()
    fn = ((y_pred < t) & (y_val == 1)).sum()
    tn = ((y_pred < t) & (y_val == 0)).sum()
    puntuaciones.append((t, tp, fp, fn, tn))

df_puntuaciones = pd.DataFrame(puntuaciones)
df_puntuaciones.columns = ['umbral', 'tp', 'fp', 'fn', 'tn']

In [ ]:
df_puntuaciones[::10]

In [ ]:
df_puntuaciones['tpr'] = df_puntuaciones.tp / (df_puntuaciones.tp + df_puntuaciones.fn)
df_puntuaciones['fpr'] = df_puntuaciones.fp / (df_puntuaciones.fp + df_puntuaciones.tn)

In [ ]:
df_puntuaciones[::10]

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_puntuaciones.umbral, df_puntuaciones.tpr, color='black', linestyle='solid', label='TPR')
plt.plot(df_puntuaciones.umbral, df_puntuaciones.fpr, color='black', linestyle='dashed', label='FPR')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.xlabel('Umbrales')
plt.title('TPR y FPR')

plt.show()

#### Baseline aleatorio

In [ ]:
def dataframe_tpr_fpr(y_val, y_pred):
    puntuaciones = []

    umbrales = np.linspace(0, 1, 101)

    for t in umbrales:
        tp = ((y_pred >= t) & (y_val == 1)).sum()
        fp = ((y_pred >= t) & (y_val == 0)).sum()
        fn = ((y_pred < t) & (y_val == 1)).sum()
        tn = ((y_pred < t) & (y_val == 0)).sum()

        puntuaciones.append((t, tp, fp, fn, tn))

    df_puntuaciones = pd.DataFrame(puntuaciones)
    df_puntuaciones.columns = ['umbral', 'tp', 'fp', 'fn', 'tn']

    df_puntuaciones['tpr'] = df_puntuaciones.tp / (df_puntuaciones.tp + df_puntuaciones.fn)
    df_puntuaciones['fpr'] = df_puntuaciones.fp / (df_puntuaciones.fp + df_puntuaciones.tn)

    return df_puntuaciones

In [ ]:
np.random.seed(1)
y_rand = np.random.uniform(0, 1, size=len(y_val))
df_rand = dataframe_tpr_fpr(y_val, y_rand)
df_rand[::10]

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_rand.umbral, df_rand.tpr, color='black', linestyle='solid', label='TPR')
plt.plot(df_rand.umbral, df_rand.fpr, color='black', linestyle='dashed', label='FPR')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.xlabel('Umbrales')
plt.title('TPR y FPR para el modelo aleatorio')

plt.show()

#### Baseline ideal

In [ ]:
num_neg = (y_val == 0).sum()
num_pos = (y_val == 1).sum()

y_ideal = np.repeat([0, 1], [num_neg, num_pos])
y_pred_ideal = np.linspace(0, 1, num_neg + num_pos)

df_ideal = dataframe_tpr_fpr(y_ideal, y_pred_ideal)
df_ideal[::10]

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_ideal.umbral, df_ideal.tpr, color='black', linestyle='solid', label='TPR')
plt.plot(df_ideal.umbral, df_ideal.fpr, color='black', linestyle='dashed', label='FPR')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.vlines(1 - y_val.mean(), -1, 2, linewidth=0.5, linestyle='dashed', color='grey')
plt.ylim(-0.03, 1.03)

plt.xlabel('Umbrales')
plt.title('TPR y FPR para el modelo ideal')

plt.show()

#### Curva ROC

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(df_puntuaciones.fpr, df_puntuaciones.tpr, color='black', label='Modelo')
plt.plot(df_rand.fpr, df_rand.tpr, color='black', lw=1,
         linestyle='dashed', alpha=0.5, label='Aleatorio')
plt.plot(df_ideal.fpr, df_ideal.tpr, color='black', lw=0.5,
         linestyle='solid', alpha=0.5, label='Ideal')

plt.legend()

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verderos Positivos')

plt.title('Curva ROC')


plt.show()

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(df_puntuaciones.fpr, df_puntuaciones.tpr, color='black')
plt.plot([0, 1], [0, 1], color='black', lw=0.7, linestyle='dashed', alpha=0.5)

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Negativos')

plt.title('Curva ROC')

plt.show()

### Usando Scikit-Learn para dibujar la curva ROC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
fpr, tpr, umbrales = roc_curve(y_val, y_pred)

In [ ]:
plt.figure(figsize=(5, 5))

plt.plot(fpr, tpr, color='black')
plt.plot([0, 1], [0, 1], color='black', lw=0.7, linestyle='dashed', alpha=0.5)

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('Tasa de Falso Positivo')
plt.ylabel('Tasa de Verdadero Positivo')

plt.title('Curva ROC')

plt.show()

#### AUC: Area bajo la curva ROC

In [ ]:
df_sub_puntuaciones = dataframe_tpr_fpr(y_val, sub_y_pred)

In [ ]:
auc(df_puntuaciones.fpr, df_puntuaciones.tpr)

In [ ]:
auc(df_sub_puntuaciones.fpr, df_sub_puntuaciones.tpr)

Comparando múltiples modelos con curvas ROC

In [ ]:
fpr_modelo1, tpr_modelo1, _ = roc_curve(y_val, y_pred)
fpr_modelo2, tpr_modelo2, _ = roc_curve(y_val, sub_y_pred)

plt.figure(figsize=(5, 5))

plt.plot(fpr_modelo1, tpr_modelo1, color='black', linestyle='solid', label='Modelo1')
plt.plot(fpr_modelo2, tpr_modelo2, color='black', linestyle='dashed', label='Modelo2')
plt.plot([0, 1], [0, 1], color='black', lw=0.7, linestyle='dashed', alpha=0.5)

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('Tasa de Falso Positivo')
plt.ylabel('Tasa de Verdadero Positivo')

plt.title('Curva ROC')
plt.legend(loc='lower right')

plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_val, y_pred)

In [ ]:
roc_auc_score(y_val, sub_y_pred)

#### Interpretación de AUC

La probabilidad de que un ejemplo positivo elegido al azar tenga una clasificación más alta que un ejemplo negativo elegido al azar. 

In [ ]:
neg = y_pred[y_val == 0]
pos = y_pred[y_val == 1]

np.random.seed(1)
neg_1 = np.random.randint(low=0, high=len(neg), size=10000)
pos_1 = np.random.randint(low=0, high=len(pos), size=10000)
(pos[pos_1] > neg[neg_1]).mean()


### Validación cruzada

In [ ]:
from utilidades import *
dibuja_validacion_cruzada()

Entrenamiento del modelo.

In [ ]:
def entrenamiento(df, y):
    cat = df[categoricas + numericas].to_dict(orient='records') 
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    modelRL = LogisticRegression(solver='liblinear')
    modelRL.fit(X, y)

    return dv, modelRL

Aplicación del modelo a nuevos datos.

In [ ]:
def predict(df, dv, modelo):
    cat = df[categoricas+ numericas].to_dict(orient='records')
    X = dv.transform(cat)

    y_pred = modelo.predict_proba(X)[:, 1]

    return y_pred

In [ ]:
from sklearn.model_selection import KFold

Ten en cuenta que al definir la división en la clase KFold, establecemos tres parámetros: 

* `n_splits = 10`: eso es K, que especifica el número de divisiones. 
* `shuffle = True`: pedimos que se mezcle los datos antes de dividirlos. 
* `random_state = 1`: debido a que hay aleatorización en el proceso (reorganización de datos), queremos que los resultados sean reproducibles, por lo que arreglamos la semilla para el generador de números aleatorios. 


In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
aucs = []

for train_idx, val_idx in kfold.split(df_train_completo):
    df_train = df_train_completo.iloc[train_idx]
    y_train = df_train.churn.values

    df_val = df_train_completo.iloc[val_idx]
    y_val = df_val.churn.values

    dv, modelo = entrenamiento(df_train, y_train)
    y_pred = predict(df_val, dv, modelo)

    rocauc = roc_auc_score(y_val, y_pred)
    aucs.append(rocauc)

In [ ]:
np.array(aucs).round(3)

Podemos obtener algunas estadísticas de esta distribución, como la media y la desviación estándar: 

In [ ]:
print('auc = %0.3f ± %0.3f' % (np.mean(aucs), np.std(aucs)))

#### Busquemos los mejores parámetros

Seeleccionamos el mejor parámetro C.

In [ ]:
def entrenamiento(df, y, C=1.0):
    cat = df[categoricas + numericas].to_dict(orient='records') 
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    modelRL = LogisticRegression(solver='liblinear')
    modelRL.fit(X, y)

    return dv, modelRL

Afinando el modelo: seleccionando el mejor parámetro C usando validación cruzada 

In [ ]:
nfolds = 5
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=1)

for C in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    aucs = []

    for train_idx, val_idx in kfold.split(df_train_completo):
        df_train = df_train_completo.iloc[train_idx]
        df_val = df_train_completo.iloc[val_idx]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, modelo = entrenamiento(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, modelo)
        
        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)

    print('C=%s, auc = %0.3f ± %0.3f' % (C, np.mean(aucs), np.std(aucs)))

Ahora entrenamos el modelo en todos los conjuntos de datos de entrenamiento y validación y aplicarlo al conjunto de datos de prueba para verificar que realmente funciona bien.


In [ ]:
y_train = df_train_completo.churn.values
y_test = df_test.churn.values

dv, modelo = entrenamiento(df_train_completo, y_train, C=0.5)
y_pred = predict(df_test, dv, modelo)

auc = roc_auc_score(y_test, y_pred)
print('auc = %.3f' % auc)

### Ejercicio

1. En este cuaderno, dibujamos el TPR y FPR para diferentes valores de un umbral, que ayudan a comprender qué significan estas métricas y también cómo cambia el rendimiento del modelo cuando elegimos un umbral diferente. Es útil hacer un ejercicio similar para la precisión y la exhaustividad, así que intenta repetir este experimento, esta vez usando  la precisión y la exhaustividad en lugar de TPR y FPR. 


In [ ]:
## Tu respuesta

2. Al dibujar la precisión y la exhaustividad para diferentes valores de umbral, podemos ver que existe un conflicto entre  la precisión y la exhaustividad: cuando una sube, la otra baja y viceversa. Esto se denomina `compensación de  la precisión y la exhaustividad`: no podemos seleccionar un umbral que haga que tanto la precisión como la exhaustividad sean buenas. Sin embargo, tenemos estrategias para seleccionar el umbral, aunque la precisión y exahustividad están en conflicto. Uno de ellos es dibujar curvas de precisión y exahustividad y ver dónde se cruzan, y usar este umbral para binarizar las predicciones. Intenta implementar esta idea.


In [ ]:
### Tu respuesta

3. Otra idea para evitar el equilibrio entre  la precisión y la exhaustividad es la puntuación F1, una puntuación que combina la  precisión y la exhaustividad en un solo valor. Luego, para seleccionar el mejor umbral, simplemente podemos elegir el que maximiza la puntuación F1. La fórmula para calcular la puntuación F1 es:  $F1 = 2\cdot P\cdot R / (P + R)$, donde $P$ es la precisión  y $R$ es la exahustividad. Implementa esta idea y selecciona el mejor umbral según la métrica F1.
 


In [ ]:
## Tu respuesta

4. Cubrimos la validación cruzada de K-fold y la usamos para comprender cómo podría verse la distribución de las puntuaciones de AUC en un conjunto de datos de prueba. Cuando $K = 10$, obtenemos 10 observaciones, que en algunas circunstancias pueden no ser suficientes. Sin embargo, la idea se puede extender a pasos repetidos de validación cruzada de K-fold. El proceso es simple: repetimos el proceso de validación cruzada de K-fold varias veces, cada vez que mezclamos el conjunto de datos de manera diferente seleccionando una semilla aleatoria diferente en cada iteración. Implementa una validación cruzada repetida y realice una validación cruzada de 10 -fold 10 veces para ver cómo se ve la distribución de las puntuaciones. 

In [ ]:
## Tu respuesta

5. Utiliza el proyecto de puntuación de clientes potenciales y el proyecto de predicción predeterminada. Prueba lo siguiente: 

 - Calcula todas las métricas que cubrimos en este cuaderno: la tabla de confusión, exactitud y recuperación, y AUC. También trata de calcular los puntajes de los ejercicios: el puntaje F1 así como el AU PR (el área bajo la curva de recuperación de exactitud). 
 - Utiliza la validación cruzada K-fold para seleccionar el mejor parámetro C para el modelo. 


In [ ]:
## Tu respuesta